In [25]:
import ee
ee.Initialize()

import json

In [26]:
input_grid = ee.FeatureCollection("users/andreasvollrath/eth_ts_analysis/ethiopia_hex_centroids_dggrid_15").sort('point_id')

In [41]:
from e_sbae.helpers.ee import processing_grid

def cascaded_extraction_ee(input_grid, config_file):

    # create a unique string
    #uuid_8str = str(uuid.uuid4())[:8]
    #export_folder = f"tmp_esbae_{uuid_8str}"

    # read config file
    with open(config_file) as f:
        config_dict = json.load(f)

    # get number of points to process
    size = input_grid.size().getInfo()
    # sort by point id
    input_grid = input_grid.sort(config_dict['ts_params']['point_id'])

    # if collection is greater than that, we iterate over chunks
    if size > 25000:
        for i in range(0, size, 25000):

            sub = ee.FeatureCollection(input_grid.toList(25000, i))
            aoi = ee.FeatureCollection(sub.geometry().convexHull(100))
            grid_fc = processing_grid(aoi, 4)
            grid = ee.FeatureCollection(grid_fc).aggregate_array('.geo').getInfo()

            # create args_list for each grid cell
            args_list = [(*l,) for l in list(enumerate(grid))]
            break
    
    return args_list

In [42]:
cascaded_extraction_ee(input_grid, 'results/config.json')

[(0,
  {'geodesic': False,
   'type': 'Polygon',
   'coordinates': [[[32.18638998368152, 7.3907604579625],
     [36.18638998368152, 7.3907604579625],
     [36.18638998368152, 11.3907604579625],
     [32.18638998368152, 11.3907604579625],
     [32.18638998368152, 7.3907604579625]]]}),
 (1,
  {'geodesic': False,
   'type': 'Polygon',
   'coordinates': [[[32.18638998368152, 11.3907604579625],
     [36.18638998368152, 11.3907604579625],
     [36.18638998368152, 15.3907604579625],
     [32.18638998368152, 15.3907604579625],
     [32.18638998368152, 11.3907604579625]]]})]

In [50]:
l = ee.List(list(range(0,24570, 1)))
#sub = ee.FeatureCollection(input_grid.toList(25000))
sub = sub.filter(ee.Filter.inList('point_id', l))
sub.size().getInfo()

0

In [51]:
sub.getInfo()

{'type': 'FeatureCollection',
 'columns': {'biome': 'Object',
  'point_id': 'Integer',
  'system:index': 'String'},
 'features': []}

In [ ]:
from e_sbae.